In [1]:
import json
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
def read_from_json(file_name):
    with open(f'{file_name}') as f:
        data = json.load(f)
    return data

In [3]:
dog_data_adoptable = read_from_json('data/dog_data_adoptable_2020.json')
cat_data_adoptable = read_from_json('data/cat_data_adoptable_2020.json')
dog_data_adopted = read_from_json('data/dog_data_adopted_2020.json')
cat_data_adopted = read_from_json('data/cat_data_adopted_2020.json')

#### Gnerating the Dataframe

In [4]:
# Functions for flatten the nested dictionar
def flatten_attr(df, attr):
    for key in df[attr][0].keys():
        df[f'{attr}_{key}'] = df[attr].apply(lambda row : row[key])

def flatten_all_attrs(df, attr_lst):
    for attr in attr_lst:
        flatten_attr(df, attr)
    return df.drop(attr_lst, axis=1)

# Display what cloumns are not picked
def get_not_included_col(df, needed_cols):
    unneed_cols = []
    for c in df.columns:
        if c not in needed_cols:
            unneed_cols.append(c)
    return unneed_cols

# Extract the state info
def extract_state_city(df):
    df['state'] = df['contact'].apply(lambda row : row['address']['state'])
    df['city'] = df['contact'].apply(lambda row : row['address']['city'])

In [5]:
# define columns that are needed and columns need to be flatten
needed_cols = ['id', 'organization_id', 'type', 'breeds', 'colors', 'age', \
               'gender', 'size', 'coat', 'attributes', 'environment',  \
               'organization_animal_id', 'status', 'published_at', 'state', 'city'
              ]
need_flatten = ['breeds', 'colors', 'attributes', 'environment']

In [6]:
def perform_flatten(df):
    df = df[needed_cols]
    df = flatten_all_attrs(df, need_flatten)
    df = df.drop_duplicates()
    return df

In [7]:
dog_df_adoptable = pd.DataFrame(dog_data_adoptable)
dog_df_adopted = pd.DataFrame(dog_data_adopted)

In [8]:
extract_state_city(dog_df_adoptable)
dog_df_adoptable = perform_flatten(dog_df_adoptable)

extract_state_city(dog_df_adopted)
dog_df_adopted = perform_flatten(dog_df_adopted)

<ipython-input-4-1cbb0db6a2c3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{attr}_{key}'] = df[attr].apply(lambda row : row[key])


In [9]:
cat_df_adoptable = pd.DataFrame(cat_data_adoptable)
cat_df_adopted = pd.DataFrame(cat_data_adopted)

In [10]:
extract_state_city(cat_df_adoptable)
cat_df_adoptable = perform_flatten(cat_df_adoptable)

extract_state_city(cat_df_adopted)
cat_df_adopted = perform_flatten(cat_df_adopted)

<ipython-input-4-1cbb0db6a2c3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{attr}_{key}'] = df[attr].apply(lambda row : row[key])


#### Some Cleaning(dog) Dorp missing values > 80%

In [11]:
def get_missing_percent(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    return pd.DataFrame({'column_name': df.columns,
                                     'percent_missing': percent_missing})


In [13]:
dog_df_adoptable_missing = get_missing_percent(dog_df_adoptable)
dog_df_adoptable_missing[dog_df_adoptable_missing['percent_missing'] > 0] 

,column_name,percent_missing
coat,coat,57.269192
organization_animal_id,organization_animal_id,43.734320
breeds_secondary,breeds_secondary,65.943302
colors_primary,colors_primary,43.715504
colors_secondary,colors_secondary,74.491972
colors_tertiary,colors_tertiary,97.547667
attributes_declawed,attributes_declawed,100.000000
environment_children,environment_children,57.934019
environment_dogs,environment_dogs,46.594330
environment_cats,environment_cats,67.316859


In [14]:
dog_df_adopted_missing = get_missing_percent(dog_df_adopted)
dog_df_adopted_missing[dog_df_adopted_missing['percent_missing'] > 0] 

,column_name,percent_missing
coat,coat,27.496894
organization_animal_id,organization_animal_id,72.869565
breeds_secondary,breeds_secondary,65.944099
colors_primary,colors_primary,27.732919
colors_secondary,colors_secondary,65.316770
colors_tertiary,colors_tertiary,95.614907
attributes_declawed,attributes_declawed,100.000000
environment_children,environment_children,42.962733
environment_dogs,environment_dogs,29.577640
environment_cats,environment_cats,66.211180


In [15]:
dog_df_adopted = dog_df_adopted.drop(['colors_tertiary', 'attributes_declawed'], axis=1)
dog_df_adopted = dog_df_adopted.fillna('missing')

In [16]:
dog_df_adoptable = dog_df_adoptable.drop(['colors_tertiary', 'attributes_declawed'], axis=1)
dog_df_adoptable = dog_df_adoptable.fillna('missing')

#### Some Cleaning(cat) Dorp missing values > 80% (keep breeds_secondary and colors_secondary)

In [17]:
cat_df_adoptable_missing = get_missing_percent(cat_df_adoptable)
cat_df_adoptable_missing[cat_df_adoptable_missing['percent_missing'] > 0] 

,column_name,percent_missing
coat,coat,54.336005
organization_animal_id,organization_animal_id,42.406029
breeds_secondary,breeds_secondary,86.977676
colors_primary,colors_primary,36.037970
colors_secondary,colors_secondary,86.586529
colors_tertiary,colors_tertiary,98.745468
environment_children,environment_children,68.651021
environment_dogs,environment_dogs,76.464415
environment_cats,environment_cats,49.370349


In [18]:
cat_df_adopted_missing = get_missing_percent(cat_df_adopted)
cat_df_adopted_missing[cat_df_adopted_missing['percent_missing'] > 0] 

,column_name,percent_missing
coat,coat,22.575509
organization_animal_id,organization_animal_id,66.039707
breeds_secondary,breeds_secondary,82.236155
colors_primary,colors_primary,16.679106
colors_secondary,colors_secondary,83.843360
colors_tertiary,colors_tertiary,97.671294
environment_children,environment_children,53.619943
environment_dogs,environment_dogs,67.094591
environment_cats,environment_cats,36.164602


In [19]:
cat_df_adoptable = cat_df_adoptable.drop(['colors_tertiary'], axis=1)
cat_df_adoptable = cat_df_adoptable.fillna('missing')

In [20]:
cat_df_adopted = cat_df_adopted.drop(['colors_tertiary'], axis=1)
cat_df_adopted = cat_df_adopted.fillna('missing')

In [21]:
# dog_df_adoptable.to_csv('dog_adoptable_2020.csv',index=False)
# dog_df_adopted.to_csv('dog_adopted_2020.csv',index=False)

# cat_df_adoptable.to_csv('cat_adoptable_2020.csv',index=False)
cat_df_adopted.to_csv('cat_adopted_2020.csv',index=False)